Learning Convolutional Neural Networks (CNNs)
- https://cdn-images-1.medium.com/max/1600/1*XbuW8WuRrAY5pC4t-9DZAQ.jpeg
- https://ai2-s2-public.s3.amazonaws.com/figures/2017-08-08/2934e9a15d0619d04ae4a4d4e2951e2ff4f45d93/21-FigureD.2-1.png

Quick Side Note:
- ReLUs https://thumbs.gfycat.com/GoodShinyGhostshrimp-size_restricted.gif

How does this differ from the neural networks we've looked at?
- Flatten
  - https://cdn-images-1.medium.com/max/1600/1*Lzx2pNLpHjGTKcofsaSH1g.png
- Convultions 
  - https://media3.giphy.com/media/i4NjAwytgIRDW/giphy.gif
  - we use "same" padding instead of "valid" padding because we want to keep shape
    - same padding puts zeros on outside
    - https://qph.fs.quoracdn.net/main-qimg-9e3419cfcd8535fb289bb1b710920d2f
- Pooling
  - optional
    - in our model we won't use it because it breaks the model/makes it not work as well
  - average pooling
    - general parts (smooth)
  - max pooling
    - sharpest parts (edges)
    - https://developers.google.com/machine-learning/practica/image-classification/images/maxpool_animation.gif

What do we have to do the code below to translate it into a functional CNN?

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing as pr 

dataset = pd.read_csv("https://ocw.mit.edu/courses/sloan-school-of-management/15-097-prediction-machine-learning-and-statistics-spring-2012/datasets/digits.csv")
datasetArray = np.array(dataset)
xfData = pr.normalize(datasetArray.T[0:-1].T)
yfData = datasetArray.T[-1]
yfData = yfData.T
yData = np.zeros((yfData.shape[0], 10))
for i in range(yfData.shape[0]):
  yData[i][yfData[i]] = 1

amountOfData = 9000
trainingData = xfData[0:amountOfData]
testingData = xfData[amountOfData:]

trainingLabels = yData[0:amountOfData]   
testingLabels = yData[amountOfData:]

model = keras.Sequential([
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(50, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit(trainingData, trainingLabels, epochs=6)

test_loss, test_acc = model.evaluate(testingData, testingLabels)

print('Test accuracy:', test_acc)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Epoch 1/6
9000/9000 [==============================] - 1s 115us/sample - loss: 0.0727 - acc: 0.4877
Epoch 2/6
9000/9000 [==============================] - 1s 98us/sample - loss: 0.0317 - acc: 0.8326
Epoch 3/6
9000/9000 [==============================] - 1s 97us/sample - loss: 0.0235 - acc: 0.8508
Epoch 4/6
9000/9000 [==============================] - 1s 94us/sample - loss: 0.0207 - acc: 0.8704
Epoch 5/6
9000/9000 [==============================] - 1s 106us/sample - loss: 0.0189 - acc: 0.8816
Epoch 6/6
1991/1991 [==============================] - 0s 79us/sample - loss: 0.0154 - acc: 0.9081
Test accuracy: 0.90808636


We have to: 
- Reshape
- Continue Normalization (It's still important)
- Remove One Hot Encoding
- Bring in idea of batches
- Look at API
  - https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l04c01_image_classification_with_cnns.ipynb#scrollTo=Gl91RPhdCaXI
  - https://keras.io/models/model/
  - https://keras.io/layers/convolutional/

2D Convolutions

In [0]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing as pr 

dataset = pd.read_csv("https://ocw.mit.edu/courses/sloan-school-of-management/15-097-prediction-machine-learning-and-statistics-spring-2012/datasets/digits.csv")
datasetArray = np.array(dataset)
xfData = pr.normalize(datasetArray.T[0:-1].T)
yfData = datasetArray.T[-1].T

amountOfData = 9000

trainingData = tf.reshape(xfData[0:amountOfData], [amountOfData, 4, 4, 1])
testingData = tf.reshape(xfData[amountOfData:], [xfData.shape[0] - amountOfData, 4, 4, 1])

trainingLabels = tf.reshape(yfData[0:amountOfData], [amountOfData, 1])   
testingLabels = tf.reshape(yfData[amountOfData:], [xfData.shape[0] - amountOfData, 1])

model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu, input_shape=(4,4,1)),
    #keras.layers.MaxPooling2D((2, 2), strides=2), Not Needed
    keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    #keras.layers.MaxPooling2D((2, 2), strides=2), This crashes it
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation=tf.nn.relu),
    keras.layers.Dense(10,  activation=tf.nn.softmax)
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',#loss='mean_squared_error', looking for categorical so this is better
              metrics=['accuracy'])

batch = 100

model.fit(trainingData, trainingLabels, epochs=10, steps_per_epoch=math.ceil(amountOfData/batch))

test_loss, test_acc = model.evaluate(testingData, testingLabels, steps=batch)

print('Test accuracy:', test_acc)

Epoch 1/10
90/90 [==============================] - 2s 27ms/step - loss: 1.0093 - acc: 0.6993
Epoch 2/10
90/90 [==============================] - 1s 14ms/step - loss: 0.1456 - acc: 0.9638
Epoch 3/10
90/90 [==============================] - 1s 14ms/step - loss: 0.0554 - acc: 0.9883
Epoch 4/10
90/90 [==============================] - 1s 14ms/step - loss: 0.0318 - acc: 0.9945
Epoch 5/10
90/90 [==============================] - 1s 14ms/step - loss: 0.0214 - acc: 0.9969
Epoch 6/10
90/90 [==============================] - 1s 14ms/step - loss: 0.0159 - acc: 0.9978
Epoch 7/10
90/90 [==============================] - 1s 14ms/step - loss: 0.0127 - acc: 0.9984
Epoch 8/10
90/90 [==============================] - 1s 14ms/step - loss: 0.0106 - acc: 0.9988
Epoch 9/10
90/90 [==============================] - 1s 14ms/step - loss: 0.0092 - acc: 0.9991
Epoch 10/10
100/100 [==============================] - 1s 6ms/step - loss: 0.0132 - acc: 0.9960
Test accuracy: 0.99598193


1D Convolutions + Dropout
- There's no difference in this versus 2D, but this is interesting cuz of dropout

In [0]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing as pr 

dataset = pd.read_csv("https://ocw.mit.edu/courses/sloan-school-of-management/15-097-prediction-machine-learning-and-statistics-spring-2012/datasets/digits.csv")
datasetArray = np.array(dataset)
xfData = pr.normalize(datasetArray.T[0:-1].T)
yfData = datasetArray.T[-1].T

amountOfData = 9000

trainingData = tf.reshape(xfData[0:amountOfData], [amountOfData, 16, 1])
testingData = tf.reshape(xfData[amountOfData:], [xfData.shape[0] - amountOfData, 16, 1])

trainingLabels = tf.reshape(yfData[0:amountOfData], [amountOfData, 1])   
testingLabels = tf.reshape(yfData[amountOfData:], [xfData.shape[0] - amountOfData, 1])

model = keras.Sequential([
    keras.layers.Conv1D(32, 3, activation=tf.nn.relu, input_shape=(16,1)),
    keras.layers.Dropout(.25),
    keras.layers.Conv1D(64, 3, padding='same', activation=tf.nn.relu),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation=tf.nn.relu),
    keras.layers.Dropout(.25),
    keras.layers.Dense(10,  activation=tf.nn.softmax)
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

batch = 100

model.fit(trainingData, trainingLabels, epochs=10, steps_per_epoch=math.ceil(amountOfData/batch))

test_loss, test_acc = model.evaluate(testingData, testingLabels, steps=batch)

print('Test accuracy:', test_acc)

Epoch 1/10
90/90 [==============================] - 2s 18ms/step - loss: 1.2503 - acc: 0.6271
Epoch 2/10
90/90 [==============================] - 1s 12ms/step - loss: 0.3342 - acc: 0.9005
Epoch 3/10
90/90 [==============================] - 1s 12ms/step - loss: 0.1808 - acc: 0.9466
Epoch 4/10
90/90 [==============================] - 1s 12ms/step - loss: 0.1250 - acc: 0.9635
Epoch 5/10
90/90 [==============================] - 1s 12ms/step - loss: 0.0982 - acc: 0.9717
Epoch 6/10
90/90 [==============================] - 1s 12ms/step - loss: 0.0807 - acc: 0.9770
Epoch 7/10
90/90 [==============================] - 1s 12ms/step - loss: 0.0689 - acc: 0.9805
Epoch 8/10
90/90 [==============================] - 1s 12ms/step - loss: 0.0608 - acc: 0.9827
Epoch 9/10
90/90 [==============================] - 1s 12ms/step - loss: 0.0535 - acc: 0.9848
Epoch 10/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0135 - acc: 0.9960
Test accuracy: 0.99598193
